**Libraries**
========

In [ ]:
!pip install accelerate==1.1.0
!pip install bitsandbytes==0.44.1
!pip install transformers==4.46.1
!pip install peft==0.13.2
!pip install trl==0.12.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 118.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━

In [ ]:
import json
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

**Hugging Face Account Config**
========

In [ ]:
from google.colab import userdata
my_secret_key = userdata.get('HF_TOKEN')
HF_TOKEN = my_secret_key

In [ ]:
LM_modelname = "meta-llama/Meta-Llama-3-8B"

**Quantization**
==========

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

**Load Tokenizer and LLM**
============

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LM_modelname,
                                          token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    LM_modelname,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

**Load Dataset**
==========

In [ ]:
import pandas as pd

# Replace with the actual paths to your validation datasets
task2_dataset = pd.read_csv("Task2-Definition-Federal_Reserve_Regulations-validation (1).csv", encoding='latin-1')

# Display the first few rows of each dataset
print("\nTask 2: Definition (Federal Reserve Regulations) Validation Dataset:")
print(task2_dataset.head())


Task 2: Definition (Federal Reserve Regulations) Validation Dataset:
                      category  \
0  Federal Reserve Regulations   
1  Federal Reserve Regulations   
2  Federal Reserve Regulations   
3  Federal Reserve Regulations   
4  Federal Reserve Regulations   

                                                term  \
0                Regulation D (Reserve Requirements)   
1          Regulation E (Electronic Funds Transfers)   
2       Regulation T (Credit by Brokers and Dealers)   
3  Regulation U (Credit by Banks for Securities P...   
4         Regulation BB (Community Reinvestment Act)   

                                              answer  
0  Governs reserve requirements for depository in...  
1  Establishes the rights and responsibilities of...  
2  Governs the extension of credit by brokers and...  
3  Governs the amount of credit banks can extend ...  
4  Implements the Community Reinvestment Act, req...  


**PEFT Training and Trainig Parameters**
==============

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# PEFT/LoRA parameters
peft_params = LoraConfig(
    r=64,  # Low-rank size
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Dropout for LoRA layers
    bias="none",  # Fine-tune only LoRA layers, no bias
    task_type="CAUSAL_LM"  # Task type is Causal Language Modeling
)

# Wrap the base model with PEFT/LoRA
peft_model = get_peft_model(model, peft_params)

training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,  # Increase epochs
    per_device_train_batch_size=8,  # Increase batch size if memory allows
    gradient_accumulation_steps=2,
    learning_rate=5e-5,  # Adjust learning rate
    weight_decay=0.01,  # Use weight decay
    warmup_ratio=0.1,  # Increase warm-up ratio
    max_grad_norm=1.0,  # Increase gradient clipping
    fp16=True,
    logging_steps=50,
    save_steps=100,
    report_to="tensorboard",
)


# Load the validation dataset
task2_path = "Task2-Definition-Federal_Reserve_Regulations-validation (1).csv"
task2_data = pd.read_csv(task2_path)

# Split the dataset into train and test sets
train_data, test_data = train_test_split(task2_data, test_size=0.2, random_state=42)

# Convert train and test datasets to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Step 2: Fine-tune on Task2 dataset (Definitions)
print("Fine-tuning on Task2: Federal Reserve Regulations Definitions...")
trainer_task2 = SFTTrainer(
    model=peft_model,  # Use the updated model from Task1
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Validation dataset for evaluation
    peft_config=peft_params,
    dataset_text_field="term",  # Update if column name differs
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)

trainer_task2.train()

# Save after Task2 fine-tuning
peft_model.save_pretrained("./task2_fine_tuned_model")
tokenizer.save_pretrained("./task2_fine_tuned_model")

print("Fine-tuning completed for Task2!")

Fine-tuning on Task2: Federal Reserve Regulations Definitions...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Step,Training Loss


Fine-tuning completed for Task2!


**Load the Trained Model**
================

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the directory of the fine-tuned model
trained_model_dir = "./task2_fine_tuned_model"

# Load the trained model and tokenizer
print("Loading the trained model...")
model = AutoModelForCausalLM.from_pretrained(trained_model_dir).to(device)
tokenizer = AutoTokenizer.from_pretrained(trained_model_dir)

print(f"Model loaded on device: {device}")

Loading the trained model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded on device: cuda


**Model Evaluation**
==============

In [ ]:
from transformers import pipeline

# Define the text-generation pipeline using LLaMA
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load the validation dataset (example: Task2)
import pandas as pd
validation_data = pd.read_csv("/content/Task2-Definition-Federal_Reserve_Regulations-validation (1).csv", encoding="latin-1")

# Extract questions, contexts, and expected answers
questions = validation_data["term"].tolist()
contexts = validation_data["category"].tolist()
expected_answers = validation_data["answer"].tolist()

# Generate predictions using structured prompts
predictions = []
print("Generating predictions...")
for question, context in zip(questions, contexts):
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"
    result = qa_pipeline(prompt, max_length=200, num_return_sequences=1)
    predictions.append(result[0]["generated_text"].strip())

# Evaluate Results
print("\nSample Results:")
for i, (question, context, pred, ref) in enumerate(zip(questions[:5], contexts[:5], predictions[:5], expected_answers[:5])):
    print(f"\nQuestion {i+1}: {question}")
    print(f"Context: {context}")
    print(f"Generated: {pred}")
    print(f"Expected: {ref}")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generating predictions...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Sample Results:

Question 1: Regulation D (Reserve Requirements)
Context: Federal Reserve Regulations
Generated: Context: Federal Reserve Regulations
Question: Regulation D (Reserve Requirements)
Answer: Regulation D (Reserve Requirements) establishes reserve requirements for depository institutions. The Federal Reserve Board determines the percentage of deposits that depository institutions must hold in reserve. This percentage is called the reserve requirement. The reserve requirement is currently 10 percent on transaction accounts and 3 percent on nontransaction accounts. The Federal Reserve Board may change the reserve requirement, but it is required to review the reserve requirement at least once every five years.
Expected: Governs reserve requirements for depository institutions, setting the minimum amount of reserves banks must hold against deposits and limits on withdrawals from savings accounts.

Question 2: Regulation E (Electronic Funds Transfers)
Context: Federal Reserve R

In [ ]:
!pip install evaluate
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00


**BERTScore Metrics**
================

In [ ]:
import evaluate

# Load the BERTScore metric
bertscore_metric = evaluate.load("bertscore")

# Example: Generated answers and expected answers
generated_answers = [
    "Regulation D governs reserve requirements for banks.",
    "Regulation E deals with electronic fund transfers."
]
expected_answers = [
    "Regulation D sets the reserve requirements for depository institutions.",
    "Regulation E establishes rules for electronic funds transfers."
]

# Step 1: Calculate BERTScore
bertscore_results = bertscore_metric.compute(
    predictions=generated_answers,
    references=expected_answers,
    lang="en"  # Specify the language
)

# Step 2: Print BERTScore results
print("\nBERTScore (Precision, Recall, F1):")
print(f"Precision: {sum(bertscore_results['precision']) / len(bertscore_results['precision']):.4f}")
print(f"Recall: {sum(bertscore_results['recall']) / len(bertscore_results['recall']):.4f}")
print(f"F1 Score: {sum(bertscore_results['f1']) / len(bertscore_results['f1']):.4f}")

# Optional: Print detailed comparisons
print("\nDetailed Results:")
for i, (gen, ref, f1) in enumerate(zip(generated_answers, expected_answers, bertscore_results["f1"])):
    print(f"Generated Answer {i + 1}: {gen}")
    print(f"Expected Answer {i + 1}: {ref}")
    print(f"BERTScore F1: {f1:.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



BERTScore (Precision, Recall, F1):
Precision: 0.9642
Recall: 0.9488
F1 Score: 0.9564

Detailed Results:
Generated Answer 1: Regulation D governs reserve requirements for banks.
Expected Answer 1: Regulation D sets the reserve requirements for depository institutions.
BERTScore F1: 0.9609
Generated Answer 2: Regulation E deals with electronic fund transfers.
Expected Answer 2: Regulation E establishes rules for electronic funds transfers.
BERTScore F1: 0.9520


**ROUGE Metrics**
==========

In [ ]:
from evaluate import load

# Load the ROUGE metric
rouge_metric = load("rouge")

# Compute ROUGE scores
rouge_scores = rouge_metric.compute(predictions=normalized_predictions, references=normalized_expected_answers)

# Print ROUGE results
print("\nROUGE Scores:")
print(rouge_scores)



ROUGE Scores:
{'rouge1': 0.1347969876616531, 'rouge2': 0.05040050805232702, 'rougeL': 0.10871257717909766, 'rougeLsum': 0.10780927254043418}


In [ ]:
from transformers import pipeline
import evaluate

# Load the ROUGE metric from the `evaluate` library
rouge_metric = evaluate.load("rouge")

# Example prompts and their expected answers
eval_data = [
    {"Term": "What is the definition of 'Federal Savings Association'?", "Definition": "A federal savings association is a financial institution chartered under section 1464 of this title."},
    {"Term": "What is the meaning of 'State Bank'?", "Definition": "A state bank refers to a bank incorporated under state law."},
    # Add more examples here
]

# Separate prompts and expected answers
prompts = [item["Term"] for item in eval_data]
expected_outputs = [item["Definition"] for item in eval_data]

# Load the pipeline for text generation
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# Generate predictions
predictions = []
print("Generating predictions...")
for prompt in prompts:
    result = text_gen_pipeline(prompt, max_length=100, num_return_sequences=1)
    predictions.append(result[0]["generated_text"].strip())

# Calculate ROUGE scores
rouge_scores = rouge_metric.compute(predictions=predictions, references=expected_outputs)

# Print Results
print("\nGenerated Predictions vs. Expected Outputs:")
for i, (prompt, pred, ref) in enumerate(zip(prompts, predictions, expected_outputs)):
    print(f"\nPrompt {i+1}: {prompt}")
    print(f"Generated: {pred}")
    print(f"Expected: {ref}")

print("\nROUGE Metrics:")
print(rouge_scores)

Generating predictions...

Generated Predictions vs. Expected Outputs:

Prompt 1: What is the definition of 'Federal Savings Association'?
Generated: What is the definition of 'Federal Savings Association'? A Federal Savings Association is a federal savings association chartered under the Federal Savings Association Act. The term is also known as a Federal Savings Bank. The Federal Savings Association Act was passed in 1933 as a response to the Great Depression. The act created the Federal Savings and Loan Insurance Corporation (FSLIC), which insured savings and loan associations. The FSLIC was abolished in 1989 and replaced by the Federal Deposit Insurance Corporation (
Expected: A federal savings association is a financial institution chartered under section 1464 of this title.

Prompt 2: What is the meaning of 'State Bank'?
Generated: What is the meaning of 'State Bank'? State Bank of India. 2. State Bank of Pakistan. 3. State Bank of Vietnam. 4. State Bank of Nigeria. 5. State Bank

**Fine Tuning**

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer

# PEFT/LoRA parameters
peft_params = LoraConfig(
    r=64,  # Low-rank size
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Dropout for LoRA layers
    bias="none",  # Fine-tune only LoRA layers, no bias
    task_type="CAUSAL_LM"  # Task type is Causal Language Modeling
)

# Wrap the base model with PEFT/LoRA
peft_model = get_peft_model(model, peft_params)

# Training parameters (common for both datasets)
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,  # Number of epochs
    per_device_train_batch_size=4,  # Batch size per GPU
    gradient_accumulation_steps=1,  # Gradient accumulation
    optim="paged_adamw_32bit",  # Optimized for large models
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,  # Learning rate
    weight_decay=0.001,
    fp16=True,  # Enable mixed precision
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Step 1: Fine-tune on fdic_task_dataset
print("Fine-tuning on FDIC Dataset...")
trainer_fdic = SFTTrainer(
    model=peft_model,  # Use the PEFT-wrapped model
    train_dataset=fdic_task_dataset,
    peft_config=peft_params,
    dataset_text_field="Definition",  # Ensure this field matches the dataset
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)

# Train the model on the FDIC dataset
trainer_fdic.train()

# Step 2: Fine-tune on terminology_task_dataset
print("Fine-tuning on Financial Terminology Dataset...")
trainer_terminology = SFTTrainer(
    model=peft_model,  # Use the updated PEFT model from FDIC fine-tuning
    train_dataset=terminology_task_dataset,
    peft_config=peft_params,
    dataset_text_field="Full Form",  # Ensure this field matches the dataset
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)

# Train the model on the Financial Terminology dataset
trainer_terminology.train()

# Save the fully fine-tuned model and tokenizer
new_model_dir = "./my_trained_model"
peft_model.save_pretrained(new_model_dir)
tokenizer.save_pretrained(new_model_dir)

print("Model and tokenizer saved successfully!")
print("Training on both datasets completed!")

Fine-tuning on FDIC Dataset...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/547 [00:00<?, ? examples/s]

Step,Training Loss
25,2.282900
50,2.667500
75,2.437800
100,2.269800
125,2.098400


Fine-tuning on Financial Terminology Dataset...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/194 [00:00<?, ? examples/s]

Step,Training Loss
25,5.404200


Model and tokenizer saved successfully!
Training on both datasets completed!


In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import Dataset
import pandas as pd

# PEFT/LoRA parameters
peft_params = LoraConfig(
    r=64,  # Low-rank size
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Dropout for LoRA layers
    bias="none",  # Fine-tune only LoRA layers, no bias
    task_type="CAUSAL_LM"  # Task type is Causal Language Modeling
)

# Wrap the base model with PEFT/LoRA
peft_model = get_peft_model(model, peft_params)

# Training parameters (common for all datasets)
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,  # Number of epochs
    per_device_train_batch_size=4,  # Batch size per GPU
    gradient_accumulation_steps=1,  # Gradient accumulation
    optim="paged_adamw_32bit",  # Optimized for large models
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,  # Learning rate
    weight_decay=0.001,
    fp16=True,  # Enable mixed precision
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Load the validation datasets
task2_path = "Task2-Definition-Federal_Reserve_Regulations-validation (1).csv"

task2_data = pd.read_csv(task2_path)

# Convert to Hugging Face datasets
task2_dataset = Dataset.from_pandas(task2_data)

# Step 2: Fine-tune on Task2 dataset (Definitions)
print("Fine-tuning on Task2: Federal Reserve Regulations Definitions...")
trainer_task2 = SFTTrainer(
    model=peft_model,  # Use the updated model from Task1
    train_dataset=task2_dataset,
    peft_config=peft_params,
    dataset_text_field="term",  # Update if column name differs
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)
trainer_task2.train()

# Save after Task2 fine-tuning
peft_model.save_pretrained("./task2_fine_tuned_model")
tokenizer.save_pretrained("./task2_fine_tuned_model")

print("Fine-tuning completed for all datasets!")

Fine-tuning on Task2: Federal Reserve Regulations Definitions...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/19 [00:00<?, ? examples/s]

Step,Training Loss


Fine-tuning completed for all datasets!


In [ ]:
from transformers import pipeline

# Define the text-generation pipeline using LLaMA
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load the validation dataset (example: Task2)
import pandas as pd
validation_data = pd.read_csv("/content/Task2-Definition-Federal_Reserve_Regulations-validation (1).csv", encoding="latin-1")

# Extract questions, contexts, and expected answers
questions = validation_data["term"].tolist()
contexts = validation_data["category"].tolist()
expected_answers = validation_data["answer"].tolist()

# Generate predictions using structured prompts
predictions = []
print("Generating predictions...")
for question, context in zip(questions, contexts):
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"
    result = qa_pipeline(prompt, max_length=200, num_return_sequences=1)
    predictions.append(result[0]["generated_text"].strip())

# Evaluate Results
print("\nSample Results:")
for i, (question, context, pred, ref) in enumerate(zip(questions[:5], contexts[:5], predictions[:5], expected_answers[:5])):
    print(f"\nQuestion {i+1}: {question}")
    print(f"Context: {context}")
    print(f"Generated: {pred}")
    print(f"Expected: {ref}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generating predictions...

Sample Results:

Question 1: Regulation D (Reserve Requirements)
Context: Federal Reserve Regulations
Generated: Context: Federal Reserve Regulations
Question: Regulation D (Reserve Requirements)
Answer: The Fed imposes reserve requirements on commercial banks to maintain a minimum amount of reserves in their accounts. The Fed also allows banks to maintain a specified amount of their reserves in the form of vault cash (i.e., cash on hand in the bank's vault). The Fed allows banks to hold a specified percentage of their reserves in the form of vault cash. The Fed also allows banks to hold a specified percentage of their reserves in the form of vault cash.
Question: Regulation Q (Interest Rates)
Answer: The Fed imposes interest rate ceilings on deposits held in commercial banks. These ceilings are designed to prevent banks from paying interest on deposits. The Fed imposes interest rate ceilings on deposits held in commercial banks. These ceilings are designed t